In [1]:
import pandas as pd

## reading in documents
dfcomments = pd.read_excel("<your-emails>.xlsx")

In [12]:

# NLP Imports
from nltk.corpus import stopwords
import nltk
 
ENGLISH_STOPWORDS = set(nltk.corpus.stopwords.words('english'))
NON_ENGLISH_STOPWORDS = set(nltk.corpus.stopwords.words()) - ENGLISH_STOPWORDS
ENGLISH_STOPWORDS.update([' ' ,'s',"/", "'s","also", "e.g.",])
STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
 
def get_language(text):
    words = set(nltk.wordpunct_tokenize(text.lower()))
    return max(((lang, len(words & stopwords)) for lang, stopwords in STOPWORDS_DICT.items()), key = lambda x: x[1])[0]

def is_english(text):
    text = text.lower()
    words = set(nltk.wordpunct_tokenize(text))
    return len(words & ENGLISH_STOPWORDS) > len(words & NON_ENGLISH_STOPWORDS)

In [8]:
dfsubset_ = dfcomments[(dfcomments["SUBJECT"].map(lambda x: is_english(str(x)))) | (dfcomments["SENDER_COMMENTS"].map(lambda x: is_english(str(x))))]
print(dfsubset_.shape)
dfsubset_["text"] = dfcomments["SUBJECT"].map(str) + ": " + dfcomments["SENDER_COMMENTS"].map(str)


(8854, 5)


/var/folders/qm/k3t85_xs2hv5bxqnxxvbxcfm0000gn/T/ipykernel_4803/121717112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsubset_["text"] = dfcomments["SUBJECT"].map(str) + ": " + dfcomments["SENDER_COMMENTS"].map(str)


In [9]:
dfsubset_ = dfcomments[(dfcomments["SUBJECT"].map(lambda x: is_english(str(x)))) | (dfcomments["SENDER_COMMENTS"].map(lambda x: is_english(str(x))))]
#dfchatbotactivitysitesearch["KEYWORD"].map(lambda x: is_english(str(x)))]
print(dfsubset_.shape)
dfsubset_["text"] = dfcomments["SUBJECT"].map(str) + ": " + dfcomments["SENDER_COMMENTS"].map(str)


(8854, 5)


/var/folders/qm/k3t85_xs2hv5bxqnxxvbxcfm0000gn/T/ipykernel_4803/2141980242.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsubset_["text"] = dfcomments["SUBJECT"].map(str) + ": " + dfcomments["SENDER_COMMENTS"].map(str)


In [15]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # # terms that will be annotated
    # annotate={"hashtag", "allcaps", "elongated", "repeated",
    #     'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
 )
# docs = dfsubset_["text"].map(lambda x: str(x).replace("_x000D_", "")).map(lambda x: " ".join(text_processor.pre_process_doc(str(x)))).dropna().tolist() 

/Users/isabelmetzger/miniconda3/envs/cxenv/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/Users/isabelmetzger/miniconda3/envs/cxenv/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
len(docs)

In [ ]:
# 8854 emails 

In [ ]:
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=35, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [ ]:
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_md")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 6 words. Make sure it is in the following format:
topic: <topic label>
"""
openai_model = OpenAI(model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "POS": pos_model
}

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(docs, embeddings)

In [ ]:
topic_model.get_topic_info()



In [ ]:
import pprint
pprint.pprint(topic_model.get_topic(1, full=True))
# or use one of the other topic representations, like KeyBERTInspired
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:4]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
topic_model.set_topic_labels(keybert_topic_labels)


In [ ]:
print(topic_model.get_topic_info())
topic_model.visualize_topics()

In [ ]:

# or ChatGPT's labels
chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["Open"].items()}
chatgpt_topic_labels[-1] = "Outlier Topic"
topic_model.set_topic_labels(chatgpt_topic_labels)

# # Reduce outliers
# new_topics = topic_model.reduce_outliers(abstracts, topics)

# # Reduce outliers with pre-calculate embeddings instead
# new_topics = topic_model.reduce_outliers(abstracts, topics, strategy="embeddings", embeddings=embeddings)
topic_model.visualize_topics(custom_labels=True)

In [ ]:
TOPICDF_1_ = topic_model.get_document_info(docs)
TOPICDF_1_

In [ ]:
print(TOPICDF_1_["CustomName"].value_counts())

In [ ]:
print(TOPICDF_1_["Name"].value_counts()[:10])

topic_model.get_topic_info()

In [ ]:
#TOPICDF_1_.to_excel("/Users/isabelmetzger/Downloads/EXPORT_WITH_TOPIC_MODEL-NEW.xlsx", index=False)

In [ ]:
# `topic_distr` contains the distribution of topics in each document
topic_distr, _ = topic_model.approximate_distribution(docs, window=4, stride=4)


In [ ]:
abstract_id = 1
print(docs[abstract_id])

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])
df


In [ ]:
# Visualize the topic-docuament distribution for a single document
topic_model.visualize_distribution(topic_distr[1], custom_labels=True)

In [ ]:
# Calculate the topic distributions on a token-level
abstract_id = 13
print(docs[abstract_id])
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs[abstract_id], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[abstract_id], topic_token_distr[0])
df

In [ ]:
# Reduce outliers
#new_topics = topic_model.reduce_outliers(docs, topics)

# # Reduce outliers with pre-calculate embeddings instead
# new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings", embeddings=embeddings)

In [ ]:
#topic_model.update_topics(docs, topics=new_topics)

In [ ]:
topic_model.get_document_info(docs)["CustomName"].value_counts()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)



In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

In [ ]:
import re
import pandas as pd
from bertopic import BERTopic

from bertopic.vectorizers import ClassTfidfTransformer
import string

def cln(i, extent=1):
    """
    String white space 'cleaner'.
    :param i:
    :param extent: 1 --> all white space reduced to length 1; 2 --> removal of all white space.
    :return:
    """

    if isinstance(i, str) and i != "":
        if extent == 1:
            return re.sub(r"\s\s+", " ", i)
        elif extent == 2:
            return re.sub(r"\s+", "", i)
    else:
        return i

transtable = str.maketrans(dict.fromkeys(string.punctuation))


def strip_punctuation(input_string):
    """cleans string by stripping punctuation """
    return input_string.translate(transtable)

#ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
model = BERTopic(#ctfidf_model=ctfidf_model,
                 verbose=True )
# Prepare data

timestamps = pd.to_datetime(dfsubset_["COMMENT_DATE"]).tolist()
# Create topics over time
model = BERTopic(verbose=True)
topics, probs = model.fit_transform(docs)
topics_over_time = model.topics_over_time(docs, timestamps)


In [ ]:

hierarchical_topics2 = model.hierarchical_topics(docs)
tree2 = model.get_topic_tree(hierarchical_topics2)
print(tree2)


In [ ]:
model.visualize_hierarchy()

In [ ]:
model.visualize_topics()

In [ ]:
# Reduce outliers
new_topics = model.reduce_outliers(docs, topics)
model.update_topics(docs, topics=new_topics)


In [ ]:
model.visualize_topics()#0, 14, 117, 74, 81, 85, 105

In [ ]:
topics_over_time = model.topics_over_time(docs, timestamps, nr_bins=20)

In [ ]:
model.get_topic_freq().head(20)

In [ ]:
model.visualize_topics_over_time(topics_over_time) #=topics_over_time, top_n_topics=20, topics=[0, 19, 9, 20, 33,37,85, 50 ,14, 117, 81, 85])

In [ ]:
model.visualize_topics_over_time(topics_over_time=topics_over_time, top_n_topics=20)

In [ ]:
docs2 = dfsubset_[dfsubset_["CATEGORY"]!="CareerOneStop"]["text"].map(lambda x: cln(str(x).replace('x000d', ' '))).tolist()
print(len(docs2))
timestamps2 = dfsubset_[dfsubset_["CATEGORY"]!="CareerOneStop"]["COMMENT_DATE"].tolist()
topics2, probs2 = model.fit_transform(docs2)
topics_over_time2 = model.topics_over_time(docs2, timestamps2, nr_bins=30)

In [ ]:
model.visualize_topics_over_time(topics_over_time=topics_over_time2)

In [ ]:
model.get_document_info(docs2).sample(2)

In [ ]:
model.get_topic_info(0) # video messages stopped

In [ ]:
TOPICDF_1_

In [53]:
TOPICDF_1_.shape

(8854, 13)

In [54]:
dfsubset_.shape

(8854, 6)

In [71]:
DFMERGED1 = dfsubset_.reset_index(drop=True).merge(TOPICDF_1_.reset_index(drop=True), right_index=True, left_index=True)

In [16]:
from __future__ import absolute_import
import string
import unicodedata
import re
import numpy as np
from collections import defaultdict
import pandas as pd
import datetime
import ujson

import warnings
 
def warning_function():
    warnings.warn("deprecated", DeprecationWarning)
 
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warning_function() 


def to_string(s):
    """Converts input to a string."""
    try:
        return str(s)
    except:
        return s.encode('utf-8')

def unicodeToAscii(s):
    """Converts unicode to ASCII."""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def create_cardinality_feature(df):
    """Creates a cardinality feature for a dataframe."""
    num_rows = len(df)
    random_code_list = np.arange(100, 1000, 1)
    return np.random.choice(random_code_list, num_rows)

def print_current_date():
    todaysdate = datetime.datetime.today().strftime('%Y-%m-%d %H:%M')
    print(f"{todaysdate}")

def get_date_timestamp():
    todaysdate = datetime.datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    return todaysdate

toeval = lambda x: eval(x) if isinstance(x, str) and x[0]=='[' else x
tolist = lambda x: x if isinstance(x, list) else (x if pd.isnull(x) else [x])

def term_in_val(val, term):
    if term in val:
        return True
    else:
        return False

def items_present_test(input_list, clist):
    return any(x in input_list for x in clist)


def chunks(l, n):
    """Yield n number of sequential chunks from l."""
    d, r = divmod(len(l), n)
    for i in range(n):
        si = (d + 1) * (i if i < r else r) + d * (0 if i < r else i - r)
        yield l[si:si + (d + 1 if i < r else d)]


flatten = lambda l: [item for sublist in l for item in sublist]

def replace_none(val, returnval = '--'):
    if val == None:
        return returnval
    
    else:
        return val
    

def matches(input1, input2):
    return (input1 == input2)

def intersection_two_lists(lst1, lst2):
    return set(lst1).intersection(lst2)


def cln(i, extent=1):
    """
    String white space 'cleaner'.
    :param i:
    :param extent: 1 --> all white space reduced to length 1; 2 --> removal of all white space.
    :return:
    """

    if isinstance(i, str) and i != "":
        if extent == 1:
            return re.sub(r"\s\s+", " ", i)
        elif extent == 2:
            return re.sub(r"\s+", "", i)
    else:
        return i

transtable = str.maketrans(dict.fromkeys(string.punctuation))


def strip_punctuation(input_string):
    """cleans string by stripping punctuation """
    return input_string.translate(transtable)

def get_match(text, rex):
    if isinstance(rex, (list, tuple, set)):
        rex = '(' + '|'.join(rex) + ')'
    result = re.findall(rex, text)
    return result

def partial_match(input_str, looking_for):
    """
    :param input_str:
    :param looking_for:
    :return:
    """
    if isinstance(input_str, str) and isinstance(looking_for, str):
        return cln(looking_for.lower(), 1) in cln(input_str.lower(), 1)
    else:
        return False


def return_no_str(val):

    if str(val).lower().strip() in ["none","nan", "n/a"]:
        return None
    else: 
        return val
    

def listify(l): return l if isinstance(l, (list, tuple, set)) else [l]

    
def flatten_json(y):
    """
    this flattens_json obj
    adds begining key to beginning
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
            out[name[:-1]] = x

    flatten(y)
    return out


def returnNonList(val):
    if not isinstance(val, list):
        return val
    if isinstance(val, list):
        if len(val) == 1:
            return val[0]
        else:
            return '|'.join(val)

    
def gather_categorical_col(df, n=10):
    df = df.applymap(returnNonList)
    categorical_col = []
    for column in df.columns:
        if df[column].dtype == object and len(df[column].unique()) <= n:
            categorical_col.append(column)

            print(f"{column} : {df[column].unique()}")
            print("====================================")
    return categorical_col


def make_inv_map(my_map):
    inv_map = {v: k for k, v in my_map.items()}
    return inv_map


def write_jsonl(file_path, lines):
    """
    wrts a .jsonl file and dump contents.
    file_path (unicode / Path): The path to the output file.
    lines (list): The JSON-serializable contents of each line.
    """
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in lines]
    Path(file_path).open('w', encoding='utf-8').write('\n'.join(data))


def make_generator(file_name, sep=","):
    """this function uses a generator"""
    qlines = (line for line in open(file_name))
    list_line = (s.rstrip().split(sep) for s in lines)
    cols = next(list_line)
    df_dicts = (dict(zip(cols, data)) for data in list_line)
    return df_dicts
import pandas as pd
import torch
from IPython import display
import jsonlines
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from transformers import AutoTokenizer, pipeline

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)


def initialize_emotion_classifier(topk=3):
    """default returns the top 3 emotions"""
    
    classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', top_k=topk)
    return classifier


def pred_emotion_classifier(classifier,
                             sample_text="I love using transformers. The best part is wide range of support and its easy to use",
                             verbose=False):

    prediction = classifier(sample_text)
    if verbose:
        print(prediction)
    return prediction



def get_text_emotion_preds(classifier, dfcomments, textcol):
    ts = get_date_timestamp()

    catpredlist = []
    dfcomments=dfcomments[dfcomments[textcol].notna()]
    N = dfcomments.shape[0]
    print(N)

    outpath=f"../{str(ts)}-{str(N)}-{str(textcol).lower().split(' ')[0]}-output-emotion-preds.jsonl"
    catpredlist = []
    with jsonlines.open(outpath, "w") as fout:
        
        for d in dfcomments.to_dict("records"):
            newd = d.copy()
            text = d[textcol]

            try: 
                if len(text) < 10: 
                    emot = classifier("NONE",)
                else:
                    emot = classifier(text,)
            except:
                pass
            
            newd['emot_topk'] = emot
            fout.write(newd)
            catpredlist.append(newd)

    return catpredlist


def turn_catpredlist_to_df(catpredlist):

    commentdfwithpreds = pd.DataFrame(catpredlist)
    
    commentdfwithpreds["emotion_top"] =commentdfwithpreds[
        'emot_topk'
    ].map(lambda x:  flatten(x)[0]if len(x) == 1  else x[0], na_action='ignore').map(lambda x: x.get('label', x))

    commentdfwithpreds["emotion_score"] =  commentdfwithpreds[
        'emot_topk'
    ].map(lambda x:  flatten(x)[0]if len(x) == 1  else x[0], na_action='ignore').map(lambda x: x.get('score', x))
    
    
    return commentdfwithpreds
    

def make_dfonlycomments(df, textcol="Anything else you\'d like to share with us?"):
    df[textcol] = df[textcol].map(return_no_str)
    print(df.shape[0])

    dfonlycomments = df[df[textcol].notna()]
    print(dfonlycomments.shape[0])
    return dfonlycomments



def get_label_val(vl):
    nv = {}
    for v in vl:
        for k, vv in v.items():
            nv[k["label"]] = vv["score"]
            
    return nv


def get_emotion_top_stats(dflistpred):
    N = dflistpred.shape[0]


    dcounts = dflistpred["emotion_top"].value_counts().reset_index().rename(columns={"emotion_top": "counts", "index": "emotion_top",
                   })
    display.display(dcounts)
    return dcounts, N



def create_group_counts_distribution(dflistpred, groupcol):
    
    dflistpred[groupcol] = dflistpred[groupcol].fillna("N/A")

    countsdf = dflistpred[groupcol].value_counts().to_frame().reset_index().rename(columns={"count": "# Responses"
                                                                                            })
    N = countsdf["# Responses"].sum()
    countsdf["% Responses"] = np.round(100*countsdf["# Responses"]/ N)

    return countsdf


def get_emotions_by_groupcol(dflistpred, groupcol):
    
    dflistpred[groupcol] = dflistpred[groupcol].fillna("Did not respond")
    
    countsemotionbygroup = pd.crosstab(
        dflistpred["emotion_top"], dflistpred[groupcol]
        ).reset_index()
    return countsemotionbygroup


negemotions = ["sadness", "anger", "fear"]
posemotions =["joy", "love"]

def filtercommments(dflistpred,emotionlist=posemotions):
    emotion_filtered_comments= (
    dflistpred[dflistpred["emotion_top"].isin(emotionlist)].sort_values(["emotion_score"], ascending=[False])
)
    return emotion_filtered_comments

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
def get_vader_sentiment(s):
    return analyzer.polarity_scores(s)

def get_top_cat(cats):
    """takes a spacy doc object and returns the category with highest score"""
    max_score = max(cats.values())
    max_cats = [k for k, v in cats.items() if v == max_score]
    max_cat = max_cats[0]
    return (max_cat, max_score)

def get_list_more(cats, score=0.4):
    l = []
    for k, v in cats.items():
        if v > score:
                
            l.append(f"{str(k)}{str(v)[:3]}")
        else:
            continue
    return '-'.join(l)

def change_2_dict(list_dicts):
    nd = {}
    for d_ in list_dicts:
        nd[d_["label"]] = d_["score"]
    return nd
    
def matches(input1, input2): return (input1 == input2)

def get_score_cats(compound_score):
    if compound_score >= 0.05:
        return "pos"
    if (-0.05 < compound_score < 0.05):
        return "neu"
    if compound_score <=-0.05:
        return "neg"
    else: return "N/A"




